In [3]:
!pip install torch transformers faiss-cpu sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 72.1 MB/s eta 0:00:00


In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle

documents = [
    "what is RAG?",
    "RAG combines Retrieval and generation",
    "open source LLM are powerful"
]

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(documents)

In [6]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [7]:
with open('faiss_index.pkl', 'wb') as f:
  pickle.dump((index, documents), f)

In [8]:
retriever = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [9]:
# load llm
!pip install transformers --upgrade --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = AutoModelForCausalLM.from_pretrained(model_name)

In [11]:
def rag_pipeline(query):
  retriever = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  query_embedding = retriever.encode([query])
  D, I = index.search(query_embedding, k=2)
  retrieved_docs = [documents[i] for i in I[0]]

  # generating responses
  input_text = query + " " + " ".join(retrieved_docs)
  inputs = tokenizer(input_text, return_tensors="pt")
  outputs = generator.generate(**inputs, max_length=100)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [13]:
# testing the pipeline

query = "what is RAG?"
print(rag_pipeline(query))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


what is RAG? what is RAG? RAG combines Retrieval and generation of information. It is a powerful tool for the development of information. It is a tool for the development of information. It is a tool for the development of information. It is a tool for the development of information. It is a tool for the development of information. It is a tool for the development of information. It is a tool for the development of information. It is a tool for the development of
